
### Top attractions and what to see in Barcelona, Spain


Introduction

Top Attractions in Barcelona

There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup and we will use it in this project. Package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

## Download, scrape and wrangle
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent to_explorer, as shown below.

In [62]:
address = 'Barcelona, Spain'

geolocator = Nominatim(user_agent="mt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona, Spain are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Barcelona, Spain are 41.3828939, 2.1774322.


#### Create a map of Matera with neighborhoods superimposed on top.

In [84]:
# create map of Matera using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.Marker(location=[latitude, longitude],popup="Barcelona")
  
    
map_barcelona

Next, we are going to start utilizing the Foursquare API to explore the venues and segment them.

#### Define Foursquare Credentials and Version

In [85]:
CLIENT_ID = 'JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP' # your Foursquare ID
CLIENT_SECRET = 'VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP
CLIENT_SECRET:VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ


#### Let's explore trending venues in Matera.

First, let's create the GET request URL. Name your URL url.

In [96]:
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
    latitude, 
    longitude,
    radius, 
    LIMIT)

# display URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JBREGZ4UNA53HX43WMAD4TQ2X2XJWMX5DPHEZEIZHQA0ACNP&client_secret=VNS40KF3V4MGSWWAV0IGQINZIGIT1EQKNCWBFPOS3QF1JMOJ&v=20180604&ll=41.3828939,2.1774322&radius=500&limit=30'

Send the GET request and examine the resutls

In [97]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebac0b5618f43001b97d7d4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'El Barri Gòtic',
  'headerFullLocation': 'El Barri Gòtic, Barcelona',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 132,
  'suggestedBounds': {'ne': {'lat': 41.3873939045, 'lng': 2.183418541112243},
   'sw': {'lat': 41.3783938955, 'lng': 2.171445858887757}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c45dcab429a0f479cb94a1e',
       'name': 'Barrio Gótico',
       'location': {'lat': 41.38366049614543,
        'lng': 2.1772899008766218,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.38366049614543,
          'lng': 2.17728990

In [98]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [74]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Barrio Gótico,Neighborhood,41.383660,2.177290
1,Gelaaati! di Marco,Ice Cream Shop,41.383186,2.177369
2,La Colmena,Dessert Shop,41.383947,2.178523
3,Pont del Carrer del Bisbe,Bridge,41.383310,2.176413
4,La Alcoba Azul,Spanish Restaurant,41.382833,2.175506
5,Frankfurt Sant Jaume,Hot Dog Joint,41.382275,2.176912
6,Plaça de Sant Felip Neri,Plaza,41.383378,2.175152
7,Zim,Wine Bar,41.383016,2.178297
8,Be Chocolat,Dessert Shop,41.382597,2.174725
9,El Cuiner de Damasc,Falafel Restaurant,41.381490,2.177677


In [101]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


# Explore Venues in Matera¶

### Let's check the size of the resulting dataframe

In [102]:
print(nearby_venues.shape)
nearby_venues

(100, 4)


,name,categories,lat,lng
0,Barrio Gótico,Neighborhood,41.383660,2.177290
1,Gelaaati! di Marco,Ice Cream Shop,41.383186,2.177369
2,La Colmena,Dessert Shop,41.383947,2.178523
3,Pont del Carrer del Bisbe,Bridge,41.383310,2.176413
4,La Alcoba Azul,Spanish Restaurant,41.382833,2.175506
5,Frankfurt Sant Jaume,Hot Dog Joint,41.382275,2.176912
6,Plaça de Sant Felip Neri,Plaza,41.383378,2.175152
7,Zim,Wine Bar,41.383016,2.178297
8,Be Chocolat,Dessert Shop,41.382597,2.174725
9,El Cuiner de Damasc,Falafel Restaurant,41.381490,2.177677


#### Let's find out how many unique categories can be curated from all the returned venues

In [103]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 48 uniques categories.


# Analyze Venues and Categories
### Next, let's group rows by category and by taking the mean of the frequency of occurrence

In [108]:
# one hot encoding
barcelona_onehot = pd.get_dummies(nearby_venues[['name']], prefix="", prefix_sep="")

# add categories column back to dataframe
barcelona_onehot['categories'] = nearby_venues['categories'] 


#move categories column to the first position
categories = barcelona_onehot['categories']
barcelona_onehot.drop(labels=['categories'], axis=1,inplace = True)
barcelona_onehot.insert(0, 'categories', categories)
barcelona_onehot.head()

,categories,Amorino,ArtiSa Barcelona,Asian Restaurant,Bacoa,Bar Belmonte,Bar Clemen-s,Bar Mariatchi,Bar Àngel,Barcelona Cooking (Cooking classes),Barcelona Duck Store,Barrio Gótico,Be Chocolat,Bidasoa,Bodega Biarritz,Bodega La Tinaja,Carabassa,Casa Llotja de Mar,Casa Lolea,Cañete,Cecconi's,Chivuo's,Circo Raluy,Clemen's,Collage Art & Cocktails Social Club,Dionisos Quick Greek,Dr. Stravinsky,El Bombón,El Cuiner de Damasc,El Quim de la Boqueria,El Xampanyet,Farmacia de la Estrella,Federal Café Gòtic,Frankfurt Sant Jaume,Frankie Gallo Cha Cha Cha,Gelaaati! di Marco,Gran Bodega Maestrazgo,H10 Cubik,H10 Port Vell,Happy Pills,Hostal OperaRamblas,Hotel 1898,Hotel Duquesa de Cardona,Impossible,Koku Kitchen Ramen,Koy Shunka,Kӕlderkold,L'antico Borgo Napoletano,La Alcoba Azul,La Burnessa,La Colmena,La Estrella,La Masala Cafe,La Plata,La Rumbeta,La Veintinueve,La Vinya del Senyor,Liceu Opera Barcelona (Gran Teatro del Liceo),Lomography Gallery Store Barcelona,Louro,L’orwell,MAG by El Magnífico,Macchina Pasta Bar,Made in Sicily,Mannà Gelats,Mercat de Sant Josep - La Boqueria,Mercat de Santa Caterina,NBA Cafe Barcelona,Nike Store,Nou Pipa Club,Nunoya,Oggi Gelato | Officina Gelato Gusto Italiano,Palau Güell (Palacio Güell),Palau de la Música Catalana (Palacio de la Música Catalana),Paradiso,Passadís del Pep,Pastelería La Estrella,Pastisseria Hofmann,Perikete,Pizza Circus,Plaça Reial (Plaza Real),Plaça de Sant Felip Neri,Plaça del Rei,Pont del Carrer del Bisbe,Pura Vida,Rabipelao,Restaurant Montiel,Rocambolesc,Sensi,Sex Shop Lovestop,Soho House Barcelona,Soho House Rooftop Pool,The Serras Hotel,The Wittmore,Tlaxcal,Topping Circus,Tucco,Viana,Vila Viniteca,Yurbban Trafalgar Hotel,Zim
0,Neighborhood,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ice Cream Shop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dessert Shop,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Spanish Restaurant,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [109]:
# let's group rows by categories and by taking the mean of the frequency of occurrence of each category

barcelona_grouped = barcelona_onehot.groupby('categories').mean().reset_index()
barcelona_grouped.head()

,categories,Amorino,ArtiSa Barcelona,Asian Restaurant,Bacoa,Bar Belmonte,Bar Clemen-s,Bar Mariatchi,Bar Àngel,Barcelona Cooking (Cooking classes),Barcelona Duck Store,Barrio Gótico,Be Chocolat,Bidasoa,Bodega Biarritz,Bodega La Tinaja,Carabassa,Casa Llotja de Mar,Casa Lolea,Cañete,Cecconi's,Chivuo's,Circo Raluy,Clemen's,Collage Art & Cocktails Social Club,Dionisos Quick Greek,Dr. Stravinsky,El Bombón,El Cuiner de Damasc,El Quim de la Boqueria,El Xampanyet,Farmacia de la Estrella,Federal Café Gòtic,Frankfurt Sant Jaume,Frankie Gallo Cha Cha Cha,Gelaaati! di Marco,Gran Bodega Maestrazgo,H10 Cubik,H10 Port Vell,Happy Pills,Hostal OperaRamblas,Hotel 1898,Hotel Duquesa de Cardona,Impossible,Koku Kitchen Ramen,Koy Shunka,Kӕlderkold,L'antico Borgo Napoletano,La Alcoba Azul,La Burnessa,La Colmena,La Estrella,La Masala Cafe,La Plata,La Rumbeta,La Veintinueve,La Vinya del Senyor,Liceu Opera Barcelona (Gran Teatro del Liceo),Lomography Gallery Store Barcelona,Louro,L’orwell,MAG by El Magnífico,Macchina Pasta Bar,Made in Sicily,Mannà Gelats,Mercat de Sant Josep - La Boqueria,Mercat de Santa Caterina,NBA Cafe Barcelona,Nike Store,Nou Pipa Club,Nunoya,Oggi Gelato | Officina Gelato Gusto Italiano,Palau Güell (Palacio Güell),Palau de la Música Catalana (Palacio de la Música Catalana),Paradiso,Passadís del Pep,Pastelería La Estrella,Pastisseria Hofmann,Perikete,Pizza Circus,Plaça Reial (Plaza Real),Plaça de Sant Felip Neri,Plaça del Rei,Pont del Carrer del Bisbe,Pura Vida,Rabipelao,Restaurant Montiel,Rocambolesc,Sensi,Sex Shop Lovestop,Soho House Barcelona,Soho House Rooftop Pool,The Serras Hotel,The Wittmore,Tlaxcal,Topping Circus,Tucco,Viana,Vila Viniteca,Yurbban Trafalgar Hotel,Zim
0,Asian Restaurant,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baby Store,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Beer Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bridge,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
num_top_venues = 1

for cat in barcelona_grouped['categories']:
    print("----"+cat+"----")
    temp = matera_grouped[matera_grouped['categories']==cat].T.reset_index()
    temp.columns = ['name','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Asian Restaurant----
               name  freq
0  Asian Restaurant   1.0


----Baby Store----
                   name  freq
0  Barcelona Duck Store   1.0


----Bar----
          name  freq
0  La Burnessa  0.33


----Beer Bar----
         name  freq
0  Kӕlderkold   1.0


----Bridge----
                        name  freq
0  Pont del Carrer del Bisbe   1.0


----Burger Joint----
    name  freq
0  Bacoa   1.0


----Café----
             name  freq
0  La Masala Cafe   1.0


----Camera Store----
                                 name  freq
0  Lomography Gallery Store Barcelona   0.5


----Candy Store----
          name  freq
0  Happy Pills   1.0


----Circus----
          name  freq
0  Circo Raluy   1.0


----Cocktail Bar----
             name  freq
0  Dr. Stravinsky   0.2


----Coffee Shop----
               name  freq
0  ArtiSa Barcelona  0.25


----Concert Hall----
                                                name  freq
0  Palau de la Música Catalana (Palacio de la Mús...   1.0


--

# 4.Map
#### Finally, let's visualize the top choices

In [111]:
#Let us map the categories to colors

colors_array = cm.rainbow(np.linspace(0, 1, len(nearby_venues['categories'])))
rainbow = [colors.rgb2hex(i) for i in colors_array]
barcelona_grouped['RGB']='red', 'blue', 'yellow','green','black','white','orange', 'maroon', 'tan', 'grey', 'pink', 'linen', 'teal', 'navy', 'plum', 'lavender', 'olive', 'indigo', 'coral', 'aqua', 'ivory', 'springgreen', 'firebrick', 'lightgrey', 'yellowgreen', 'tomato', 'oldlace', 'blueviolet', 'cadetblue', 'lightsalmon', 'orchid', 'chocolate', 'peru', 'skyblue', 'gold', 'palegreen', 'hotpink', 'lime', 'seashell', 'salmon', 'cornsilk', 'purple', 'aliceblue', 'mistyrose', 'bisque', 'burlywood', 'steelblue', 'khaki'
barcelona_grouped

,categories,Amorino,ArtiSa Barcelona,Asian Restaurant,Bacoa,Bar Belmonte,Bar Clemen-s,Bar Mariatchi,Bar Àngel,Barcelona Cooking (Cooking classes),Barcelona Duck Store,Barrio Gótico,Be Chocolat,Bidasoa,Bodega Biarritz,Bodega La Tinaja,Carabassa,Casa Llotja de Mar,Casa Lolea,Cañete,Cecconi's,Chivuo's,Circo Raluy,Clemen's,Collage Art & Cocktails Social Club,Dionisos Quick Greek,Dr. Stravinsky,El Bombón,El Cuiner de Damasc,El Quim de la Boqueria,El Xampanyet,Farmacia de la Estrella,Federal Café Gòtic,Frankfurt Sant Jaume,Frankie Gallo Cha Cha Cha,Gelaaati! di Marco,Gran Bodega Maestrazgo,H10 Cubik,H10 Port Vell,Happy Pills,Hostal OperaRamblas,Hotel 1898,Hotel Duquesa de Cardona,Impossible,Koku Kitchen Ramen,Koy Shunka,Kӕlderkold,L'antico Borgo Napoletano,La Alcoba Azul,La Burnessa,La Colmena,La Estrella,La Masala Cafe,La Plata,La Rumbeta,La Veintinueve,La Vinya del Senyor,Liceu Opera Barcelona (Gran Teatro del Liceo),Lomography Gallery Store Barcelona,Louro,L’orwell,MAG by El Magnífico,Macchina Pasta Bar,Made in Sicily,Mannà Gelats,Mercat de Sant Josep - La Boqueria,Mercat de Santa Caterina,NBA Cafe Barcelona,Nike Store,Nou Pipa Club,Nunoya,Oggi Gelato | Officina Gelato Gusto Italiano,Palau Güell (Palacio Güell),Palau de la Música Catalana (Palacio de la Música Catalana),Paradiso,Passadís del Pep,Pastelería La Estrella,Pastisseria Hofmann,Perikete,Pizza Circus,Plaça Reial (Plaza Real),Plaça de Sant Felip Neri,Plaça del Rei,Pont del Carrer del Bisbe,Pura Vida,Rabipelao,Restaurant Montiel,Rocambolesc,Sensi,Sex Shop Lovestop,Soho House Barcelona,Soho House Rooftop Pool,The Serras Hotel,The Wittmore,Tlaxcal,Topping Circus,Tucco,Viana,Vila Viniteca,Yurbban Trafalgar Hotel,Zim,RGB
0,Asian Restaurant,0.000000,0.00,1.0,0.0,0.000,0.0,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.0,0.000,0.000000,red
1,Baby Store,0.000000,0.00,0.0,0.0,0.000,0.0,0.000000,0.000,0.000000,1.0,0.0,0.000000,0.0,0.000,0.000,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.0,0.000,0.000000,blue
2,Bar,0.000000,0.00,0.0,0.0,0.000,0.0,0.333333,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.333333,0.0,0.000,0.000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000000,0.0,0.0,0.0,0.000,0.000000,yellow
3,Beer Bar,0.000000,0.00,0.0,0.0,0.000,0.0,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000,0.0,0.0

In [113]:
# create map
map_visitbarcelona = folium.Map(location=[latitude, longitude], zoom_start=16)

for lat, lon, poi, cluster, col in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories'],barcelona_grouped['RGB']):
    label=folium.Popup(' Name: ' + str(poi) + ' Category: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=col,
        fill=True,
        fill_color=col,
        fill_opacity=0.8).add_to(map_visitmatera)
        
map_visitbarcelona.add_child(folium.LayerControl())
map_visitbarcelona